In [88]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy
from functools import partial
import os
import yaml
import torch
from guided_diffusion.unet import create_model,create_model_ir
from guided_diffusion.gaussian_diffusion import create_sampler
from util.logger import get_logger
import cv2
import numpy as np
from skimage.io import imsave
import warnings
import shutil
import matplotlib.pyplot as plt
def calculate_ssim(imageA, imageB):
    assert imageA.shape == imageA.shape
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(grayA, grayB, full=True)
    return score

def calculate_qabf(imageA, imageB):
    return 0.0

def calculate_vif(imageA, imageB):
    return 0.0
    
# MI (Mutual Information)
def calculate_mi(imageA, imageB):
    hist_2d, _, _ = np.histogram2d(imageA.ravel(), imageB.ravel(), bins=20)
    pxy = hist_2d / float(np.sum(hist_2d))
    px = np.sum(pxy, axis=1)
    py = np.sum(pxy, axis=0)
    px_py = px[:, None] * py[None, :]
    nzs = pxy > 0
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))
    return mutual_information(hist_2d)

def calculate_sd(image):
    return np.std(image)

def calculate_en(image):
    return shannon_entropy(image)
def image_read(path, mode='RGB'):
    img_BGR = cv2.imread(path).astype('float32')
    assert mode == 'RGB' or mode == 'GRAY' or mode == 'YCrCb', 'mode error'
    if mode == 'RGB':
        img = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2RGB) # OpenCV 中用于将图像的颜色空间从 BGR（蓝-绿-红）转换为 RGB（红-绿-蓝）的函数
    elif mode == 'GRAY':  
        img = np.round(cv2.cvtColor(img_BGR, cv2.COLOR_BGR2GRAY))
    elif mode == 'YCrCb':
        img = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2YCrCb)
    return img






In [89]:
def plot(a,b,c):
    # 创建子图
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    # 显示原始RGB图像
    axs[0].imshow(a)
    axs[1].imshow(b)
    axs[2].imshow(c)
    # 显示图像
    plt.show()
    
def adjust_image_shape(img, scale):
    h, w = img.shape[:2]
    h = h - h % scale
    w = w - w % scale
    return img[:h, :w, :]

def gray_to_rgb(gray_img):
    # 将灰度图像的单通道数据复制到三个通道中
    return np.stack([gray_img] * 3, axis=-1)

In [90]:
vi_img_folder = "/home/2021/hexin/XJ2/D3FM/input/vi"
ir_img_folder = "/home/2021/hexin/XJ2/D3FM/input/ir"
f_img_folder = "/home/2021/hexin/XJ2/D3FM/output/recon"
name = "FLIR_08248"
img_name = f"{name}.jpg"
f_name = f"{name}.png"
print(f"list = {os.path.join(vi_img_folder,img_name)}")

mode = "GRAY" #RGB:[1,1,H,W,3] GRAY:[1,1,H,W]
vi = image_read(os.path.join(vi_img_folder,img_name),mode=mode)[np.newaxis,np.newaxis, ...]/255.0 
ir = image_read(os.path.join(ir_img_folder,img_name), mode=mode)[np.newaxis,np.newaxis, ...]/255.0 
f = image_read(os.path.join(f_img_folder,f_name), mode=mode)[np.newaxis,np.newaxis, ...]/255.0 


vi = np.squeeze(vi)  
ir = np.squeeze(ir) 
f = np.squeeze(f) 
print(f"vi.shape = {vi.shape}")
#plot(vi,ir,f)

# crop to make divisible








list = /home/2021/hexin/XJ2/D3FM/input/vi/FLIR_08248.jpg
vi.shape = (311, 481)


In [91]:
if(mode == "GRAY"):
    vi = gray_to_rgb(vi)
    ir = gray_to_rgb(ir)
    f = gray_to_rgb(f)
    
scale = 32
ir = adjust_image_shape(ir,scale)
vi = adjust_image_shape(vi,scale)
f = adjust_image_shape(f,scale)
assert ir.shape == vi.shape

print(ir.shape)
print(f.shape)

inf_img = ir.squeeze()
vis_img = vi.squeeze()
f_img = f.squeeze()
#plot(vis_img,inf_img,f_img)

(288, 480, 3)
(288, 480, 3)


In [92]:
ssim_value1 = calculate_ssim(f, vi)
ssim_value2 = calculate_ssim(f, vi)

qabf_value = calculate_qabf(f, vi)
vif_value = calculate_vif(f, vi)
mi_value = calculate_mi(f, vi)
sd_value = calculate_sd(f)
en_value = calculate_en(f)

print(f"SSIM_1: {ssim_value1}, SSIM_2: {ssim_value2}")
print(f"Qabf: {qabf_value}")
print(f"VIF: {vif_value}")
print(f"MI: {mi_value}")
print(f"SD: {sd_value}")
print(f"EN: {en_value}")

SSIM_1: 0.8152705009436524, SSIM_2: 0.8152705009436524
Qabf: 0.0
VIF: 0.0
MI: 0.7175706252035706
SD: 0.1431395560503006
EN: 6.921187224464151
